In [37]:
def loadCoreArticle(string):    
    f = open('/home/lr/Downloads/conference_data/' + string + '.txt','r')
    sample = f.readlines()

    dict_finallis_keyword = eval(sample[0])
    f.close()


    return dict_finallis_keyword
"""
tochi_core = loadCoreArticle('tochi_core')
avi_core = loadCoreArticle('avi_core')
icmi_core = loadCoreArticle('icmi_core')
gi_core = loadCoreArticle('gi_core')
chi_core = loadCoreArticle('chi_core')
idc_core = loadCoreArticle('idc_core')
lak_review = loadCoreArticle('lak_review')
"""

cof = ['assets','avi','chi','cscw','dis','gi','group','icmi','idc','its','iui','lak','ls','mobilechi','tochi','ubicomp','uist']

In [38]:
import copy
title_list = []
reference_list = {}
for s in cof:
    name = s + "_core"
    
    data = loadCoreArticle(name)
    #print (s)
    #print (len(data))
    title_list.extend(list(data.keys()))
    for i in data:
        reference_list[i] = copy.deepcopy(data[i])

In [39]:
title_list[8] = ": designing a new class of computational toys"
reference_list[title_list[8]] = reference_list['curlybot: designing a new class of computational toys']
title_list[16] = ": Haptic Feedback to Enhance Early Reading"
reference_list[title_list[16]] = reference_list['FeelSleeve: Haptic Feedback to Enhance Early Reading']
title_list[32] = ": The Effects of Curriculum-Aligned Making on Children's Self-Identity"
reference_list[title_list[32]] = reference_list["I Make, Therefor I Am: The Effects of Curriculum-Aligned Making on Children's Self-Identity"]

In [40]:
#获取数据
from elasticsearch import helpers
import elasticsearch


total_text = {}
total_bibliometrics = {}
total_index_term = {}
total_reference = {}
total_author = {}
article_author = {}
total_article = {}
total_id = {}
total_conf = {}
total_year = {}

ES_SERVERS = [{
    # 'host': 'localhost',
    'host': '127.0.0.1',
    'port': 9200
}]

es_client = elasticsearch.Elasticsearch(
    hosts=ES_SERVERS
)

es_result = helpers.scan(
    client=es_client,
    query={
    "query": {
        "match_all": {}
    }
},
    scroll='5m',
    index='conference_total',
    #doc_type='conference_total',
    # timeout='1m'
)
# print(len(es_result))

#h_heads=[]
cont=0
for i in es_result:
    data_year = i['_source']
    h_head = i['_source']['h_head']
    id_num = i['_id']
    year = data_year['h_year']
    if year not in total_article.keys():
        total_article[year] = {'title':[], 'abstract':[]}
    #if 'group' not in data_year.keys():
        #print (h_head)
       

    

           
    author = data_year['author']
    #reference = data_year['reference']
    index_term = data_year['index_term']
    bibliometrics = data_year['citition']
    title = data_year['title']
    abstract = data_year['abstract']
    if title not in title_list:
        continue
        
    #if title == "StudentLife: assessing mental health, academic performance and behavioral trends of college students using smartphones":
        #print (bibliometrics)
        #print (cont)
        
    #print (title)
    #print (cont)
    #print (id_num)
    cont += 1 
    total_id.setdefault(year,[]).append(id_num)
    total_bibliometrics.setdefault(year,[]).append(bibliometrics)
    total_reference.setdefault(year,[]).append(reference_list[title])
    total_index_term.setdefault(year,[]).append(index_term)
    total_article[year]['title'].append(title)
    total_article[year]['abstract'].append(abstract)
    article_author.setdefault(year,[]).append(author)
    total_conf.setdefault(year,[]).append(h_head)
 

    
author_data = article_author
print (cont)

164


# 对比共同作者

In [6]:
import copy
#初始化列表
num_same_author = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_author.setdefault(s,[]).append(copy.deepcopy(single))
        
#print (num_same_author)

for s in author_data:
    count = 0
    for i in author_data[s]:
        for j in i:
            name1 = j['name']       
            
            for m in author_data:
                temp = -1
                for n in author_data[m]:
                    temp += 1
                    for o in n:
                        name2 = o['name']
                        
                        if name1 == name2:
                            num_same_author[s][count][m][temp] += 1
                            
                           
        count += 1

# 对比共同标签

In [7]:
import copy
#初始化列表
num_same_index = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_index.setdefault(s,[]).append(copy.deepcopy(single))
        
for s in total_index_term:
    count = 0
    for i in total_index_term[s]:
        #print (i)
        for j in i:    
            
            for m in total_index_term:
                temp = -1
                for n in total_index_term[m]:
                    temp += 1
                    for o in n:                        
                        if j == o:
                            num_same_index[s][count][m][temp] += 1
                            
                           
        count += 1


# 对比共同关键词

In [147]:
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
import numpy as np
import re
import math

#设置stopwords，缩略词拆解

stopwords = set (stopwords.words('english')+list(punctuation)+[').'])


#分段成句和分句成词
def splitSentence(paragraph):
    sent = sent_tokenize(paragraph)
    return sent


def wordtokenizer(sentence):
    word = []
    words = WordPunctTokenizer().tokenize(sentence)
    word = word + words
    return word

#文章预处理
def standardization(word_sent):
#转换为小写字母    
    text = []
    for s in word_sent:
        text.append(s.lower())
    
    return text

#词形还原
def lemmatizer(word):
    wnl = WordNetLemmatizer()
    text = []
#对单词类型标注
    text_tag = np.array(nltk.pos_tag(word))
    #print (text_tag)
    
    for s in text_tag:
        if s[1].startswith('N'):
            if s[1] == 'NNS' or s[1] == 'NN':
                text.append(wnl.lemmatize(s[0],'n'))
            else:
                text.append(s[0])
        elif s[1].startswith('V'):
            text.append(wnl.lemmatize(s[0], 'v'))
        elif s[1].startswith('J'):
            text.append(wnl.lemmatize(s[0], 'a'))
        elif s[1].startswith('R'):
            text.append(wnl.lemmatize(s[0], 'r'))
        else:
            text.append(s[0])
            
    #print (text)
    new_text = [word for word in text if word not in stopwords and 3<len(word)]
    
    return new_text

def filtrated(word):

    wnl = WordNetLemmatizer()
    text = []
#对单词类型标注
    text_tag = np.array(nltk.pos_tag(word))
    #print (text_tag)
    
    for s in text_tag:
        if s[1].startswith('N'):            
            text.append(s[0])
        
    
    return text

#统计词频
def wordFreq(word):
    word_Freq = {}
    for s in word:
        if not s in word_Freq:
            word_Freq[s] = 1
        else:
            word_Freq[s] += 1
    
    num = 0
    for key,value in word_Freq.items():
        num = num + value
    
    return word_Freq, num


In [148]:
year_list = []
for s in total_bibliometrics:
    year_list.append(s)
print (year_list)

['2017', '2016', '2015', '2014', '2013', '2012', '2011', '2008', '2006', '2010', '2004', '2009', '2000', '2018', '2005', '2001', '2007', '1998', '1997']


In [170]:
#1st提取关键词
from sklearn.feature_extraction.text import CountVectorizer
education_keywords = ['education','learn','teach','educate','exam','classroom','student','curriculum','course','school','campus','college']
online_learning = ['online','mooc']
stem = ['program','app','programming']
interface = ['interface','interact','interaction','agent','tangible']

keyword_sum = {}
count = 0
for s in year_list:
    for i in range(len(total_article[s]['title'])):
        if total_article[s]['abstract'][i] != []:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i] + " " + total_article[s]['abstract'][i]
        else:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i]
            
        sent = splitSentence(text) 
        
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word
        #print (word_list)
        word_standard = standardization(word_list)
        #print (word_standard)
        word_ori = lemmatizer(word_standard)
        
        single = filtrated(word_ori)
        word_Freq, num = wordFreq(single)
        
        temp = []
        #print (word_Freq)
        
        group_count = {'online': 0,'stem':0,'interface':0,'student':0}
        for j in word_Freq:
            if word_Freq[j] > 1 and j not in education_keywords:
                for m in range(word_Freq[j]):
                    temp.append(j)

        """
        if max(group_count, key = group_count.get) == 'student' and group_count['student'] == 1:
            #print (count)
            if count in high_cited:
                temp = ['student'] * 20
            else:                
                temp.append('student')"""
        """elif max(group_count, key = group_count.get) == 'online' and group_count['online'] > 0:
            temp = ['online'] * group_count['online']
        elif max(group_count, key = group_count.get) == 'stem' and group_count['stem'] > 0:
            temp = ['stem'] * group_count['stem']
        elif max(group_count, key = group_count.get) == 'interface' and group_count['interface'] > 0:
            temp = ['interface'] * group_count['interface']
        else:
            temp = []"""
        #print (total_article[s]['title'][i])
        #if total_article[s]['title'][i] == "To block or not to block, that is the question: students' perceptions of blocks-based programming":
            #print (temp)
            
        
        keyword_sum.setdefault(s,[]).append(temp)
        #print (temp)
        count += 1

In [10]:
#提取关键词
from sklearn.feature_extraction.text import CountVectorizer
education_keywords = ['education','learn','teach','educate','exam','classroom','student','curriculum','course','school','campus','college']
online_learning = ['online','mooc']
stem = ['program','app','programming']
interface = ['interface','interact','interaction','agent','tangible']

keyword_sum = {}
count = 0
for s in year_list:
    for i in range(len(total_article[s]['title'])):
        if total_article[s]['abstract'][i] != []:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i] + " " + total_article[s]['abstract'][i]
        else:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i]
            
        sent = splitSentence(text) 
        
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word
        #print (word_list)
        word_standard = standardization(word_list)
        #print (word_standard)
        word_ori = lemmatizer(word_standard)
        
        single = filtrated(word_ori)
        word_Freq, num = wordFreq(single)
        
        temp = []
        #print (word_Freq)
        
        group_count = {'online': 0,'stem':0,'interface':0,'student':0}
        for j in word_Freq:
            if word_Freq[j] > 1 and j not in education_keywords:
                if j in online_learning:
                    group_count['online'] += 1
                    for m in range(word_Freq[j]):
                        temp.append(j)
                if j in stem:
                    group_count['stem'] += 1
                    for m in range(word_Freq[j]):
                        temp.append(j)
                if j in interface:
                    group_count['interface'] += 1
                    if count in high_cited:
                        for m in range(word_Freq[j]*20):
                            temp.append("interface")
                    else:
                        for m in range(word_Freq[j]):
                            temp.append("interface")
                else:
                    for m in range(word_Freq[j]):
                        temp.append(j)
                        
            if j == 'student':
                group_count['student'] = 1
        """
        if max(group_count, key = group_count.get) == 'student' and group_count['student'] == 1:
            #print (count)
            if count in high_cited:
                temp = ['student'] * 20
            else:                
                temp.append('student')"""
        """elif max(group_count, key = group_count.get) == 'online' and group_count['online'] > 0:
            temp = ['online'] * group_count['online']
        elif max(group_count, key = group_count.get) == 'stem' and group_count['stem'] > 0:
            temp = ['stem'] * group_count['stem']
        elif max(group_count, key = group_count.get) == 'interface' and group_count['interface'] > 0:
            temp = ['interface'] * group_count['interface']
        else:
            temp = []"""
        #print (total_article[s]['title'][i])
        #if total_article[s]['title'][i] == "To block or not to block, that is the question: students' perceptions of blocks-based programming":
            #print (temp)
            
        
        keyword_sum.setdefault(s,[]).append(temp)
        #print (temp)
        count += 1

NameError: name 'high_cited' is not defined

In [207]:
#final提取关键词
from sklearn.feature_extraction.text import CountVectorizer
education_keywords = ['education','learn','teach','educate','exam','classroom','student','curriculum','course','school','campus','college']
online_learning = ['online','mooc']
stem = ['program','app','programming']
interface = ['interface','interact','interaction','agent','tangible']
student = ['mining','student','learner']
keyword_sum = {}
count = 0
for s in year_list:
    for i in range(len(total_article[s]['title'])):
        if total_article[s]['abstract'][i] != []:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i] + " " + total_article[s]['abstract'][i]
        else:
            text = total_article[s]['title'][i] + ' ' + total_article[s]['title'][i]
            
        sent = splitSentence(text) 
        
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word
        #print (word_list)
        word_standard = standardization(word_list)
        #print (word_standard)
        word_ori = lemmatizer(word_standard)
        
        single = filtrated(word_ori)
        word_Freq, num = wordFreq(single)
        
        temp = []
        #print (word_Freq)
        #if total_article[s]['title'][i] == "SmartGPA: how smartphones can assess and predict academic performance of college students":
            #print (single)
        group_count = {'online': 0,'stem':0,'student':0,'interface':0}
        for j in word_Freq:
            if word_Freq[j] > 1 and j not in education_keywords:
                if j in online_learning:
                    group_count['online'] += 1
                    for m in range(word_Freq[j]):
                        temp.append("online")
                if j in stem:
                    group_count['stem'] += 1
                    for m in range(word_Freq[j]):
                        temp.append("stem")
                if j in interface:
                    group_count['interface'] += 1
                    if count in high_cited:
                        for m in range(word_Freq[j]*8):
                            temp.append("interface")
                    else:
                        for m in range(word_Freq[j]):
                            temp.append("interface")
                else:
                    for m in range(word_Freq[j]):
                        temp.append(j)
                        
            if j in student:
                group_count['student'] = 1
            elif j in online_learning:
                group_count['online'] += 1
            elif j in stem:
                group_count['stem'] += 1
            elif j in interface:
                group_count['interface'] += 1
                
        if max(group_count, key = group_count.get) == 'student' and group_count['student'] == 1:
            #print (count)
            if count in high_cited:
                temp = ['student'] * 20
            else:                
                temp.append('student')
        elif max(group_count, key = group_count.get) == 'online' and group_count['online'] > 0:
            if count in high_cited:
                temp = ['online'] * 20
            else:
                temp.append('online')
        elif max(group_count, key = group_count.get) == 'stem' and group_count['stem'] > 0:
            if count in high_cited:
                temp = ['stem'] * 20  
            else:
                temp.append('stem')
        elif max(group_count, key = group_count.get) == 'interface' and group_count['interface'] > 0:
            if count in high_cited:
                temp = ['interface'] * 20
            else:
                temp.append('interface')
        
            
        #print (total_article[s]['title'][i])
        if total_article[s]['title'][i] == "Learning analytics and educational data mining: towards communication and collaboration":
            print (group_count)
            print (temp)
            
        
        keyword_sum.setdefault(s,[]).append(temp)
        #print (temp)
        count += 1

{'online': 0, 'stem': 0, 'student': 1, 'interface': 0}
['analytics', 'analytics', 'analytics', 'analytics', 'analytics', 'data', 'data', 'data', 'data', 'data', 'mining', 'mining', 'mining', 'mining', 'towards', 'towards', 'communication', 'communication', 'communication', 'collaboration', 'collaboration', 'collaboration', 'increase', 'increase', 'research', 'research', 'research', 'method', 'method', 'community', 'community', 'student']


In [208]:
import copy
#初始化列表
num_same_keyword = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_keyword.setdefault(s,[]).append(copy.deepcopy(single))
        
for s in keyword_sum:
    count = 0
    for i in keyword_sum[s]:
        for j in i:    
            
            for m in keyword_sum:
                temp = -1
                for n in keyword_sum[m]:
                    temp += 1
                    for o in n:                        
                        if j == o:
                            num_same_keyword[s][count][m][temp] += 1
                            break
                            
                           
        count += 1


# 对比共同参考文献

In [209]:
import copy
#初始化列表
num_same_ref = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        num_same_ref.setdefault(s,[]).append(copy.deepcopy(single))
        

for s in total_reference:
    count = 0
    for i in total_reference[s]:
        #print (i)
        for j in i['reference']:    
            
            for m in total_reference:
                temp = -1
                for n in total_reference[m]:
                    temp += 1
                    for o in n['reference']:                        
                        if j == o:
                            num_same_ref[s][count][m][temp] += 1
                            break
                            
                           
        count += 1
        
        

# 引用网络

In [75]:
node_list = []
count = 0
total_year = {}
for s in total_reference:
    for i in total_reference[s]:
        #print (i)
        total_year[count] = s
        for j in i['reference']:    
            
            for m in total_reference:
                temp = -1
                for n in total_reference[m]:
                    temp += 1
                    for o in n['reference']:                        
                        if j == o:
                            node_list.append(j)
                            break
                            
                           
        count += 1
        
node_list = list(set(node_list))
print (len(node_list))

5157


In [93]:
count_id = {}
total = 0
for s in total_reference:
    #print (s)
    #print (total_reference[s])
    for i in total_reference[s]:
        #print (i)
        for j in i['reference']:
            #print (i)
            count = 200
            for m in node_list:
                if j == m:
                    count_id.setdefault(count,[]).append(total)
                    break
                count += 1
                
        total += 1

            
for s in count_id:
    count_id[s] = list(set(count_id[s]))

2017
2016
2015
2014
2013
2012
2011
2008
2006
2010
2004
2009
2000
2018
2005
2001
2007
1998
1997


In [ ]:
total = 0
for s in total_reference:
    for i in range(len(total_reference[s])):
        if total == 
        

In [94]:
core_citing = {}
for s in count_id:
    #print (s)
    #print (count_id[s])
    for i in count_id[s]:
        core_citing.setdefault(i,[]).append(s)
        
print (len(core_citing))

164


In [95]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns = ['Source','Target','id','Type','weight'])
index = 0
Type = 'Undirected'
for s in count_id:
    for i in count_id[s]:
        temp = []
        if i in count_id.keys() and s in count_id[i]:
            print (s)
            print (i)
            continue
        
        #source = i
        #target = s
        
        temp.append(i)
        temp.append(s)
        temp.append(index)
        temp.append(Type)
        temp.append(1)
        
        df.loc[index] = temp
        
        index += 1
        

In [96]:
df.to_csv('edge_small.csv', sep=',', header=True, index=False)

In [107]:
import math

df_node = pd.DataFrame(columns = ['id','Label','size','age'])
index = 0
for s in count_id:    
    temp = []
    #source = i
    #target = s
    
    weight = math.log(len(count_id[s]))
    

    temp.append(s)
    temp.append('')
    temp.append(weight)
    temp.append(30)
    
   

    df_node.loc[index] = temp

    index += 1

for s in core_citing:
    if s not in count_id.keys():
        temp = []
        weight = math.log(len(core_citing[s]))
        year = total_year[s]
        
        age = 2020 - int(year) + 1
        #print (age)
        
        
        temp.append(s)
        
        if int(s) in onlineLearning:
            temp.append("Online Learning" + str(s))
        elif int(s) in tabletopInterface:
            temp.append('Interface' + str(s))
        elif int(s) in STEMedu:
            temp.append('STEM' + str(s))
        elif int(s) in stuAnalysis:
            temp.append('analysis' + str(s))
        else:
            temp.append('')
        
        temp.append(weight)
        temp.append(age)
        #print (len(temp))
        
        df_node.loc[index] = temp
        index += 1

In [108]:
print (df_node.shape)

(5321, 4)


In [106]:
for s in df_node['age']:
    print (s)
    

In [109]:
df_node.to_csv('node_small.csv', sep=',', header=True, index=False)

# 计算相似得分

In [210]:
import copy
#初始化列表
sim_score = {}
single = {}
for s in author_data:
    #print (len(author_data[s]))
    single[s] = [0] * len(author_data[s])
#print (single)

for s in author_data:
    for i in author_data[s]:
        sim_score.setdefault(s,[]).append(copy.deepcopy(single))
a = 0
b = 0
c = 0
d = 1

for s in total_article:
    for i in range(len(total_article[s]['title'])):
        
        for year in num_same_author[s][i]:
            for j in range(len(num_same_author[s][i][year])):
                same_ref = num_same_ref[s][i][year][j]
                same_author = num_same_author[s][i][year][j]
                same_index = num_same_index[s][i][year][j]
                same_keyword = num_same_keyword[s][i][year][j]
                
                sim_score[s][i][year][j] = a*same_ref + d*same_keyword + c*same_index + b*same_author  
                
                
        


# 生成力导向数据

In [211]:
import pandas as pd
import numpy as np
df = pd.DataFrame(columns = np.arange(164))
article_position = {}
num = 0
for s in sim_score:
    #print (s)
    count = 0
    for i in sim_score[s]:
        title = total_article[s]['title'][count]
        article_position[title] = num
        temp = []
        for j in i:
            temp.extend(i[j])
        #print (temp)
        df.loc[num] = temp
        
        count += 1
        num += 1
#print (df)

In [212]:
df_edge = pd.DataFrame(columns = ['Source','Target','id','Type','weight'])
index = 0
Type = 'Undirected'

for s in df:
    for i in df:
        temp = []
        if s != i and df[s][i] > 15 and i > s:
            temp.append(i)
            temp.append(s)
            temp.append(index)
            temp.append(Type)
            temp.append(df[s][i])
            
            #print (temp)

            df_edge.loc[index] = temp

            index += 1

In [213]:
print (df_edge.shape)

(286, 5)


In [175]:
import math

df_node = pd.DataFrame(columns = ['id','Label','size','age'])
index = 0

for s in range(len(new_cite_list)):
    temp = []
    #print (new_cite_list[s])
    temp.append(s)
    
    if int(s) in onlineLearning:
        temp.append("Online Learning" + str(s))
    elif int(s) in tabletopInterface:
        temp.append('Interface' + str(s))
    elif int(s) in STEMedu:
        temp.append('STEM' + str(s))
    elif int(s) in stuAnalysis:
        temp.append('analysis' + str(s))
    else:
        temp.append('')
        
    temp.append(new_cite_list[s])
    
    num = df1.loc[s]
    age = 2020 - int(num['year']) + 1
    temp.append(age)
    
    df_node.loc[index] = temp

    index += 1


In [167]:
print (df_node.shape)

(164, 4)


In [214]:
df_edge.to_csv('edge_small_final.csv', sep=',', header=True, index=False)
df_node.to_csv('node_samll.csv', sep=',', header=True, index=False)

In [238]:
t = []
for s in df:
    #if s > 100:
        #break
    for i in df:
        #if i > 100:
            #break
        if s != i and df[s][i] > 15 and i > s:
            print (str(s) + ' -- ' + str(i) + " {weight: " + str(df[s][i]/20) + "}")
            t.append(s)
            t.append(i)
s2 = list(set(t))

0 -- 19 {weight: 0.9}
2 -- 62 {weight: 0.8}
3 -- 52 {weight: 0.85}
3 -- 75 {weight: 1.0}
5 -- 36 {weight: 1.1}
6 -- 145 {weight: 0.9}
6 -- 156 {weight: 0.85}
9 -- 62 {weight: 1.0}
15 -- 68 {weight: 0.95}
16 -- 37 {weight: 0.8}
18 -- 44 {weight: 1.0}
21 -- 75 {weight: 0.85}
22 -- 68 {weight: 3.0}
22 -- 151 {weight: 3.0}
22 -- 161 {weight: 2.0}
23 -- 37 {weight: 0.95}
23 -- 83 {weight: 0.85}
23 -- 136 {weight: 1.05}
27 -- 68 {weight: 3.0}
27 -- 151 {weight: 3.0}
27 -- 161 {weight: 2.0}
31 -- 37 {weight: 0.8}
32 -- 36 {weight: 0.9}
39 -- 68 {weight: 0.9}
39 -- 69 {weight: 1.2}
44 -- 68 {weight: 3.0}
44 -- 151 {weight: 3.0}
44 -- 161 {weight: 2.0}
45 -- 156 {weight: 1.0}
46 -- 68 {weight: 3.0}
46 -- 77 {weight: 0.8}
46 -- 151 {weight: 3.1}
46 -- 161 {weight: 2.0}
47 -- 68 {weight: 3.9}
47 -- 77 {weight: 0.95}
47 -- 151 {weight: 3.0}
47 -- 161 {weight: 2.0}
49 -- 68 {weight: 0.95}
50 -- 68 {weight: 1.05}
56 -- 68 {weight: 3.0}
56 -- 151 {weight: 3.0}
56 -- 156 {weight: 1.0}
56 -- 161 {weigh

In [170]:
"""for s in range(164):
    if s not in s2:
        print (s)"""
print (len(s2))

37


In [20]:
t = []
node_temp = {}
edges_temp = []
count = 0
for s in df:
    #if s > 100:
        #break
    for i in df:
        #if i > 100:
            #break
        if s != i and df[s][i] > 10:
            if s not in node_temp.keys():
                node_temp[s] = count
                count += 1
            if i not in node_temp.keys():
                node_temp[i] = count
                count += 1
            edges_temp.append({"source": s, "target": i, "weight": df[s][i]/2})
                
s2 = list(set(t))

In [23]:
node = []
edge = []
for s in node_temp:
    node.append({"name:": s})
    
print (node)
for s in edges_temp:
    source = s["source"]
    target = s['target']
    edge.append({"source": node_temp[source], "target": node_temp[target], "weight": s['weight']})
    print (edge)

[{'name:': 11}, {'name:': 143}, {'name:': 64}, {'name:': 76}, {'name:': 65}, {'name:': 84}, {'name:': 68}, {'name:': 77}, {'name:': 85}, {'name:': 113}, {'name:': 108}, {'name:': 116}, {'name:': 138}, {'name:': 139}]
[{'source': 0, 'target': 1, 'weight': 7.0}, {'source': 2, 'target': 3, 'weight': 11.0}, {'source': 4, 'target': 5, 'weight': 7.0}, {'source': 6, 'target': 7, 'weight': 12.0}, {'source': 3, 'target': 2, 'weight': 11.0}, {'source': 7, 'target': 6, 'weight': 12.0}, {'source': 5, 'target': 4, 'weight': 7.0}, {'source': 8, 'target': 9, 'weight': 6.0}, {'source': 10, 'target': 11, 'weight': 12.0}, {'source': 9, 'target': 8, 'weight': 6.0}, {'source': 11, 'target': 10, 'weight': 12.0}, {'source': 12, 'target': 13, 'weight': 6.0}, {'source': 13, 'target': 12, 'weight': 6.0}, {'source': 1, 'target': 0, 'weight': 7.0}]


In [ ]:
print (df[95])

# 修正引用数

In [121]:
now = 2020
index = 4
high_cited = {}
missing = {}
new_cite_list = []
year_list = []
for s in total_bibliometrics:
    year_list.append(s)
    #print (total_bibliometrics[s])
    count = 0
    if s == "2014/2015":
        year = 2014.0
    else:
        year = float(s)
    for i in total_bibliometrics[s]:
        #print (i)

        new_cite = index / (now - year + 1) * (i + 1)
        #print (new_cite)
        new_cite_list.append(new_cite)
        #if s == '2014':
            #print (len(new_cite_list))
print (len(new_cite_list))

164


In [123]:
color = ['#97CBFF','#2894FF','#005AB5']


In [124]:
high_cited = []

for s in range(len(new_cite_list)):
    if new_cite_list[s] > 40:
        print (str(s) + "{color:" + color[2] + "}")
        high_cited.append(s)
        continue
    if new_cite_list[s] > 30:
        print (str(s) + "{color:" + color[1] + "}")
        high_cited.append(s)
        continue
    if new_cite_list[s] > 20:
        print (str(s) + "{color:" + color[0] + "}")
        
        continue
print (high_cited)    

2{color:#97CBFF}
30{color:#97CBFF}
39{color:#97CBFF}
40{color:#97CBFF}
52{color:#97CBFF}
61{color:#97CBFF}
65{color:#97CBFF}
69{color:#005AB5}
71{color:#97CBFF}
72{color:#97CBFF}
77{color:#97CBFF}
81{color:#97CBFF}
83{color:#97CBFF}
84{color:#005AB5}
85{color:#005AB5}
88{color:#97CBFF}
91{color:#97CBFF}
92{color:#97CBFF}
96{color:#2894FF}
97{color:#97CBFF}
103{color:#97CBFF}
104{color:#2894FF}
107{color:#97CBFF}
131{color:#97CBFF}
132{color:#97CBFF}
137{color:#97CBFF}
150{color:#2894FF}
153{color:#97CBFF}
155{color:#005AB5}
159{color:#97CBFF}
161{color:#2894FF}
[69, 84, 85, 96, 104, 150, 155, 161]


# 导出文章信息

In [44]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame(columns = ['index','title','abstract','author','citation','conference','year','id'])

In [45]:
for s in total_article:
    for i in range(len(total_article[s]['title'])):
        temp = []
        title = total_article[s]['title'][i]
        abstract = total_article[s]['abstract'][i]
        author = author_data[s][i]
        index = total_id[s][i]
        conf = total_conf[s][i]
        citation = total_bibliometrics[s][i]
        num = article_position[title]
        
        temp.append(num)
        temp.append(title)
        temp.append(abstract)
        temp.append(author)
        temp.append(citation)
        temp.append(conf)
        temp.append(s)
        temp.append(index)
        
        
        df1.loc[num] = temp
        
        num += 1
        
print (df1)

    index                                              title  \
142   142  Blocks4All: Overcoming Accessibility Barriers ...   
143   143  “Bursting the Assistance Bubble”: Designing In...   
144   144  Bots & (Main)Frames: Exploring the Impact of T...   
145   145  Science Everywhere: Designing Public, Tangible...   
146   146  ConceptScape: Collaborative Concept Mapping fo...   
147   147  Collaborative Live Media Curation: Shared Cont...   
148   148  Co-designing online privacy-related games and ...   
149   149  EmotiW 2018: Audio-Video, Student Engagement a...   
0       0  Learning to Code in Localized Programming Lang...   
1       1  Enabling Real-Time Adaptivity in MOOCs with a ...   
2       2  Writing Reusable Code Feedback at Scale with M...   
3       3  Intelligent tutors as teachers' aides: explori...   
4       4  Trends and issues in student-facing learning a...   
5       5  Follow the successful crowd: raising MOOC comp...   
6       6  Enabling Collaboration in Lea

In [101]:
onlineLearning = [69,87,96,37,68,88,36,39,54,55,59,60,63,71,82,89,91,74]
stuAnalysis = [84,65,104,102,103,111,131,149,85,113,56,57]
STEMedu = [156,52,59,2,24,34,133,14,51,6,81,145]
tabletopInterface = [151,161,77,105,132,137,159,98,108,116,117,129,138,139,135,157,163]

In [56]:
def print_id(onlineLearning):
    temp = []
    for s in onlineLearning:
        #print (s)
        num = df1.loc[s]
        #print (num['id'])
        #print (df['index'][num])
        temp.append(num['id'])

    print (temp)
    
print_id(onlineLearning)
print_id(tabletopInterface)
print_id(STEMedu)
print_id(stuAnalysis)

['352', '600', '16891', '283', '392', '612', '287', '285', '16979', '16980', '18011', '17885', '17966', '350', '18977', '5917', '5744', '544']
['8925', '12689', '3241', '11999', '15208', '10606', '11208', '22393', '11908', '12489', '23147', '22752', '10632', '15478', '18295', '11170', '14752']
['11110', '14959', '18011', '74', '6570', '17747', '16394', '20230', '7252', '1732', '13840', '8181']
['24154', '23470', '685', '661', '681', '999', '9013', '21556', '610', '1003', '16992', '17907']


{'2018': {'title': ['Blocks4All: Overcoming Accessibility Barriers to Blocks Programming for Children with Visual Impairments', '“Bursting the Assistance Bubble”: Designing Inclusive Technology with Children with Mixed Visual Abilities', 'Bots & (Main)Frames: Exploring the Impact of Tangible Blocks and Collaborative Play in an Educational Programming Game', 'Science Everywhere: Designing Public, Tangible Displays to Connect Youth Learning Across Settings', 'ConceptScape: Collaborative Concept Mapping for Video Learning', 'Collaborative Live Media Curation: Shared Context for Participation in Online Learning', 'Co-designing online privacy-related games and stories with children', 'EmotiW 2018: Audio-Video, Student Engagement and Group-Level Affect Prediction'], 'abstract': ['Blocks-based programming environments are a popular tool to teach children to program, but they rely heavily on visual metaphors and are therefore not fully accessible for children with visual impairments. We evalua

In [52]:
import os
path = "/home/lr/Downloads/conference_data/education_high_cited/"
if not os.path.exists(path):
     os.mkdir(path)
#date_rand =time.strftime("%Y-%m-%d",time.localtime())
file_name = path+'artical_info.xlsx'
writer = pd.ExcelWriter(file_name)

df1.to_excel(writer, "article_info")
writer.save()

In [21]:
import xlrd
path = "/home/lr/Downloads/conference_data/education_high_cited/"
file_name = path+'artical_info.xlsx'


df = pd.DataFrame(pd.read_excel(file_name))
df=df.set_index(['index'])
print(df)

       Unnamed: 0                                              title  \
index                                                                  
39             39          Quiet interfaces that help students think   
40             40  On the design of Camelot, an outdoor game for ...   
41             41  roBlocks: a robotic construction kit for mathe...   
129           129      : designing a new class of computational toys   
130           130  Alice: lessons learned from building a 3D syst...   
83             83  An Intelligent Interface for Learning Content:...   
84             84  Framing Feedback: Choosing Review Environment ...   
85             85  Programming, Problem Solving, and Self-Awarene...   
86             86  'Don't Waste My Time': Use of Time Information...   
87             87  Facilitator, Functionary, Friend or Foe?: Stud...   
88             88  Learn Piano with BACh: An Adaptive Learning In...   
89             89  Improving Real-Time Captioning Experiences fo

# 聚类分组+词云

In [243]:
#初步分类
onlineLearning = [23,96,61,90,62,83,54,15,5,45,147,49,50,63,82,72,47,69,68,77,59,146,31,159,134]
tabletopInterface = [117,98,108,129,116,139,138]
STEMedu = [1,40,34,52,11,94,137,64,76,122,6,142,143,24,92,156,151,158,125,140]

In [5]:
#最终分类
onlineLearning = [94,87,69,88,91,54,36,60,63,39,67,71,82,146,147,42,33,40,25,17,5,49,50,43,32,0,16,18,66,74]
tabletopInterface = [151,161,68,132,77,105,133,137,148,163,98,129,78,123,150,128,157,116,110,112,130,108,95,124,139,122,70,72,40,22,27,114,46,56,90]
STEMedu = [156,52,24,2,55,59,34,6,145,9,62,53,142,1,152,85,45,76,140,125,141,113,143,11]
stuAnalysis = [84,37,65,104,81,102,131,103,155,80,79,93,86,92,41,57,79,4,13,75,21,3,31,19,28,30,35,7,20,38,8,58,10]


In [116]:
"""def generateWC(group):
    word_sum = []
    for s in group:
        position = findPosition(s)
        year = position[0]
        index = position[1]

        title = total_article[year]['title'][index]
        abstract = total_article[year]['title'][index]
        text = title + ' ' + title + ' ' + abstract

        sent = splitSentence(title) 
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word

        word_standard = standardization(word_list)
        word_ori = lemmatizer(word_standard)

        word_sum.extend(word_ori)

    #print (word_sum)
    prep_list = '/'.join(word_sum)
    #print (prep_list)
    
    from wordcloud import WordCloud
    wc = WordCloud(width = 600, height = 300).generate(prep_list)
    image = wc.to_image()
    image.show()"""

In [244]:
def wordList(group):
    word_sum = []
    for s in group:
        info = df1.loc[s]
        title = info['title']
        abstract = info['abstract']

        text = title + ' ' + title + ' ' + abstract

        sent = splitSentence(title) 
        word_list = []
        for m in sent:
            word = wordtokenizer(m)                  
            word_list = word_list + word

        word_standard = standardization(word_list)
        word_ori = lemmatizer(word_standard)

        word_sum.extend(word_ori)

    prep_list = '/'.join(word_sum)
    return prep_list

ol_list = wordList(onlineLearning)
ti_list = wordList(tabletopInterface)
stem_list = wordList(STEMedu)
sa_list = wordList(stuAnalysis)

In [247]:
print (stem_list)

learn/code/localized/programming/language/uncover/trajectory/informal/learning/large/online/community/creator/remixing/pathway/computational/thinking/block/block/question/student/perception/block/base/programming/teach/program/gamified/semantics/maker/movement/fablabs/education/experience/technology/research/compare/tangible/graphical/programming/language/informal/science/education/computational/thinking/computational/making/interest/textile/analyze/make/gendered/perspective/play/sound/maker/embody/metaphor/help/child/learn/enable/collaboration/learn/computer/program/inclusive/child/vision/impairment/blocks4all/overcoming/accessibility/barrier/block/program/child/visual/impairment/main/frame/explore/impact/tangible/block/collaborative/play/educational/programming/game/programming/problem/solving/self/awareness/effect/explicit/guidance/trace/base/framework/analyze/synthesize/educational/progression/storytelling/alice/motivates/middle/school/girl/learn/computer/program/extend/tangible/in

In [141]:
def generateWC(prep_list,string):
    from wordcloud import WordCloud
    wc = WordCloud(width = 800, height = 600).generate(prep_list)
    image = wc.to_image()
    wc.to_file(string)
    image.show()
    
generateWC(ol_list,'OnlingLearning.png')
generateWC(ti_list,'tabletopInterface.png')
generateWC(stem_list,'STEM.png')
generateWC(sa_list,'StudentAnalysis.png')

In [193]:
for s in range(len(new_cite_list)):
    if new_cite_list[s] < 30 and new_cite_list[s] > 20:
        print (s)
        continue
    """if new_cite_list[s] > 30:
        print (str(s) + "{color:" + color[1] + "}")
        continue
    if new_cite_list[s] > 20:
        print (str(s) + "{color:" + color[0] + "}")
        continue"""

2
6
24
29
34
36
39
54
55
59
60
63
71
77
81
82
89
91
97
102
103
105
111
131
132
133
137
145
149
154
158


In [143]:
for s in range(142,150):
    if s in onlineLearning:
        print (s)
        print ('OL')
    elif s in tabletopInterface:
        print (s)
        print ('Interface')
    elif s in STEMedu:
        print (s)
        print ('stem')
    elif s in stuAnalysis:
        print (s)
        print ('sa')

142
stem
143
stem
145
stem
146
OL
147
OL
148
Interface


In [154]:
def annualCount(stuAnalysis):
    year_data = {}
    for s in stuAnalysis:
        #print (s)
        #print(df1['year'][s])
        year = df1['year'][s]
        if year not in year_data.keys():
            year_data[year] = 1
        else:
            year_data[year] += 1
    print(year_data)
    
    
annualCount(onlineLearning)
annualCount(tabletopInterface)
annualCount(STEMedu)
annualCount(stuAnalysis)

{'2013': 4, '2014': 6, '2015': 10, '2018': 2, '2016': 6, '2017': 2}
{'2005': 2, '1997': 2, '2014': 5, '2004': 3, '2012': 3, '2009': 2, '2018': 1, '2013': 3, '2010': 2, '2006': 2, '2007': 1, '2011': 3, '2008': 1, '2015': 3, '2016': 2}
{'2007': 1, '2015': 6, '2016': 2, '2017': 5, '2018': 3, '2001': 1, '2013': 1, '2014': 1, '2000': 2, '2006': 1, '2011': 1}
{'2014': 6, '2015': 6, '2012': 3, '2004': 1, '2007': 1, '2013': 3, '2017': 6, '2016': 7}
